In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your data
essays = pd.read_csv('../llm-detect-ai-generated-text/train_essays.csv')
prompts = pd.read_csv('../llm-detect-ai-generated-text/train_prompts.csv')

# Merge essays with prompts based on 'prompt_id'
data = essays.merge(prompts, on='prompt_id', how='left')
data['combined_text'] = data['prompt_name'] + " " + data['instructions'] + " " + data['text']

# Split data into train and validation sets
train_data, val_data = train_test_split(data, test_size=0.3, random_state=42)


In [2]:
from transformers import DebertaTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

# Initialize tokenizer
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')  

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True
        )
        return {
            'input_ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Creating the dataset
train_dataset = TextDataset(train_data['combined_text'].tolist(), train_data['generated'].tolist(), tokenizer)
val_dataset = TextDataset(val_data['combined_text'].tolist(), val_data['generated'].tolist(), tokenizer)

# DataLoader setup
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [3]:
from transformers import DebertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# Model initialization
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "mps" if torch.backends.mps.is_available() else device
print(f'Using device: {device}')
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=2)  
model.to(device)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_loader) * 3  # num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop with accuracy tracking
for epoch in range(3):  # num_epochs
    model.train()
    total_train_loss = 0
    total_train_correct = 0
    total_train_examples = 0

    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_train_loss += loss.item()

        preds = torch.argmax(outputs.logits, dim=-1)
        total_train_correct += (preds == batch['labels']).sum().item()
        total_train_examples += batch['labels'].size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    train_accuracy = total_train_correct / total_train_examples
    print(f'Epoch {epoch+1}, Train Loss: {total_train_loss / len(train_loader)}, Train Accuracy: {train_accuracy}')

    # Validation step
    model.eval()
    total_val_loss = 0
    total_val_correct = 0
    total_val_examples = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            total_val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=-1)
            total_val_correct += (preds == batch['labels']).sum().item()
            total_val_examples += batch['labels'].size(0)

    val_accuracy = total_val_correct / total_val_examples
    print(f'Validation Loss: {total_val_loss / len(val_loader)}, Validation Accuracy: {val_accuracy}')


Using device: mps


/opt/miniconda3/envs/llm-detect-deberta/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/miniconda3/envs/llm-detect-deberta/lib/python3.9/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation tor

Epoch 1, Train Loss: 0.03849066208306246, Train Accuracy: 0.983402489626556
Validation Loss: 0.0190664929407201, Validation Accuracy: 0.9975845410628019
Epoch 2, Train Loss: 0.01433093804136796, Train Accuracy: 0.9979253112033195
Validation Loss: 0.01777570607821242, Validation Accuracy: 0.9975845410628019
Epoch 3, Train Loss: 0.00984539467868326, Train Accuracy: 0.9979253112033195
Validation Loss: 0.017166739428760663, Validation Accuracy: 0.9975845410628019


In [63]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 model for paraphrasing
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')
t5_model.eval()
t5_model.to(device)

def paraphrase_text(texts, t5_model, tokenizer, max_length=200, batch_size=8):
    """ Function to generate paraphrased text using T5 in batches """
    paraphrased_texts = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        input_ids = tokenizer(batch_texts, max_length=max_length, return_tensors="pt").input_ids.to(t5_model.device)
        paraphrased_ids = t5_model.generate(input_ids, max_length=max_length, num_return_sequences=1, num_beams=2)
        for j, g in enumerate(paraphrased_ids):
            try:
                paraphrased_text = tokenizer.decode(g, skip_special_tokens=True)
                if not paraphrased_text:  # Check if paraphrased text is empty
                    raise ValueError("Empty paraphrase generated")
            except:
                paraphrased_text = batch_texts[j]  # Fallback to the original text if paraphrasing fails
            paraphrased_texts.append(paraphrased_text)
    return paraphrased_texts

def paraphrase_and_prepare_data(validation_data, model, tokenizer):
    """ Function to paraphrase and prepare data for DataLoader """
    
    # Ensure text entries are valid strings
    val_data_reset = validation_data.reset_index(drop=True)
    
    # Paraphrase the 'text' part of the validation data
    paraphrased_texts = paraphrase_text(validation_data['text'].tolist(), model, tokenizer)

    # Add paraphrased text as a new column in the validation data
    paraphrased_texts_df = pd.DataFrame({'text': paraphrased_texts})

    # Combine the paraphrased text with the prompt and instructions
    val_data_reset['paraphrased_text'] = val_data_reset['prompt_name'].fillna('').str.cat(paraphrased_texts_df['text'].fillna(''), sep=' ')
    
    return val_data_reset

# Apply paraphrasing and preparation to the validation data
# paraphrased_val_data = paraphrase_and_prepare_data(val_data, t5_model, t5_tokenizer)

# # Use your existing TextDataset class for the paraphrased data
# paraphrased_val_dataset = TextDataset(paraphrased_val_data['paraphrased_text'].tolist(), paraphrased_val_data['generated'].tolist(), tokenizer)

# # DataLoader for the paraphrased validation dataset
# paraphrased_val_loader = DataLoader(paraphrased_val_dataset, batch_size=8, shuffle=False)

paraphrased_val_data = paraphrase_and_prepare_data(val_data, t5_model, t5_tokenizer)
paraphrased_val_dataset = TextDataset(paraphrased_val_data['paraphrased_text'].tolist(), paraphrased_val_data['generated'].tolist(), tokenizer)
paraphrased_val_loader = DataLoader(paraphrased_val_dataset, batch_size=8, shuffle=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [64]:
# Evaluate existing dataset against adversarial attacks
def evaluate_model(model, dataloader):
    model.eval()
    total_correct = 0
    total_examples = 0
    model.to(device)

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            preds = torch.argmax(outputs.logits, dim=-1)
            total_correct += (preds == batch['labels']).sum().item()
            total_examples += batch['labels'].size(0)

    accuracy = total_correct / total_examples
    return accuracy

# Evaluate your DeBERTa model on the paraphrased validation data
accuracy = evaluate_model(model, paraphrased_val_loader)
print(f'Accuracy on Paraphrased Data: {accuracy}')

Accuracy on Paraphrased Data: 0.9975845410628019
